In [1]:
!git clone https://github.com/DuyNguDao/ST-GCN-Pytorch.git

Cloning into 'ST-GCN-Pytorch'...
remote: Enumerating objects: 94, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 94 (delta 3), reused 16 (delta 0), pack-reused 66 (from 1)
Receiving objects: 100% (94/94), 113.03 MiB | 47.49 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [ ]:
import os
import time
import torch
import pickle
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')
from tqdm import tqdm
from torch.utils import data
from torch.optim.adadelta import Adadelta
import sys
sys.path.append('/kaggle/working/ST-GCN-Pytorch')
from models.stgcn import *
from torch.utils.data import DataLoader, TensorDataset
from collections import OrderedDict
import logging
import yaml
from dataloader.dataset import processing_data
import datetime

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# clear memory cuda
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

# Get parameter
with open("/kaggle/input/yaml-action-pkl/config.yaml", "r") as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print("Error loading YAML:", exc)
        config = {}

# parameter
input_dataset_train = config.get('dataset-path-train', '')
input_dataset_test = config.get('dataset-path-test', '')
epochs = config.get('epochs', 50)
batch_size = config.get('batch-size', 32)
input_size = config.get('img-size', 224)
num_frame = config.get('num-frame', 15)
path_save_model = config.get('project', 'saved_models')
classes_name = config.get('classes', ['Fall Down', 'Lying Down', 'Sit down', 'Sitting', 'Stand up', 'Standing', 'Walking'])

print("Class name:", classes_name)

features, labels = [], []
# Load dataset train
with open(input_dataset_train, 'rb') as f:
    fts, lbs = pickle.load(f)
    features.append(fts)
    labels.append(lbs)
del fts, lbs

labels = np.concatenate(labels, axis=0)
features = np.concatenate(features, axis=0)
features = features[:, ::2, :, :]
features[:, :, :, :2] = processing_data(features[:, :, :, :2])
x_train = features
y_train = labels

features, labels = [], []
with open(input_dataset_test, 'rb') as f:
    fts, lbs = pickle.load(f)
    features.append(fts)
    labels.append(lbs)
del fts, lbs

labels = np.concatenate(labels, axis=0)
features = np.concatenate(features, axis=0)
features = features[:, ::2, :, :]
features[:, :, :, :2] = processing_data(features[:, :, :, :2])
x_valid = features
y_valid = labels

del features, labels

train_dataset = TensorDataset(torch.tensor(x_train, dtype=torch.float32).permute(0, 3, 1, 2),
                              torch.tensor(y_train, dtype=torch.float32))
val_dataset = TensorDataset(torch.tensor(x_valid, dtype=torch.float32).permute(0, 3, 1, 2),
                            torch.tensor(y_valid, dtype=torch.float32))

del x_train, x_valid, y_train, y_valid

# create folder save
if not os.path.exists(path_save_model):
    os.mkdir(path_save_model)
count = 0
while os.path.exists(path_save_model + f'/exp{count}'):
    count += 1
path_save_model = path_save_model + f'/exp{count}'
os.mkdir(path_save_model)

# load data loader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=batch_size, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=batch_size, pin_memory=True)

del train_dataset, val_dataset

def set_training(model, mode=True):
    for p in model.parameters():
        p.requires_grad = mode
    model.train(mode)
    return model

graph_args = {'strategy': 'spatial'}
model = TwoStreamSpatialTemporalGraph(graph_args, len(classes_name)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
losser = torch.nn.BCELoss()

def train_model(model, losser, optimizer, num_epochs):
    best_loss_acc = -1
    for epoch in range(num_epochs):
        losses_train = 0.0
        train_corrects = 0
        model = set_training(model, True)
        pbar_train = tqdm(train_loader, desc=f'Epoch {epoch}', unit='batch')
        for batch_vid, labels in pbar_train:
            mot = batch_vid[:, :2, 1:, :] - batch_vid[:, :2, :-1, :]
            mot, batch_vid, labels = mot.to(device), batch_vid.to(device), labels.to(device)
            outputs = model((batch_vid, mot))
            loss = losser(outputs, labels)
            model.zero_grad()
            loss.backward()
            optimizer.step()
            losses_train += loss.item()
            _, preds = torch.max(outputs, 1)
            train_corrects += (preds == labels.data.argmax(1)).detach().cpu().numpy().mean()

        epoch_loss = losses_train / len(train_loader)
        epoch_acc = train_corrects / len(train_loader)
        logging.warning(f'Train: Accuracy: {epoch_acc}, Loss: {epoch_loss}')

        # Validation
        losses_val = 0.0
        val_corrects = 0
        model = set_training(model, False)
        with torch.no_grad():
            for batch_vid, labels in val_loader:
                mot = batch_vid[:, :2, 1:, :] - batch_vid[:, :2, :-1, :]
                mot, batch_vid, labels = mot.to(device), batch_vid.to(device), labels.to(device)
                outputs = model((batch_vid, mot))
                loss = losser(outputs, labels)
                losses_val += loss.item()
                _, preds = torch.max(outputs, 1)
                val_corrects += (preds == labels.data.argmax(1)).detach().cpu().numpy().mean()

            epoch_loss = losses_val / len(val_loader)
            epoch_acc = val_corrects / len(val_loader)
            logging.warning(f'Validation: Accuracy: {epoch_acc}, Loss: {epoch_loss}')
            if best_loss_acc == -1 or best_loss_acc <= epoch_acc:
                best_loss_acc = epoch_acc
                torch.save(model.state_dict(), path_save_model + '/best.pt')
                logging.warning(f'Saved best model at epoch {epoch}')
    return model

def main():
    model_trained = train_model(model, losser, optimizer, num_epochs=epochs)
    torch.save(model_trained.state_dict(), path_save_model + '/last.pt')
    logging.warning(f'Saved last model at {path_save_model}/last.pt')
    print("Complete !")

if __name__ == '__main__':
    main()


Class name: ['Fall Down', 'Lying Down', 'Sit down', 'Sitting', 'Stand up', 'Standing', 'Walking']


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Epoch 0:   0%|          | 0/887 [00:00<?, ?batch/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Epoch 49: 100%|██████████| 887/887 [0

Complete !


In [3]:
import os
import time
import torch
import pickle
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')
from tqdm import tqdm
from torch.utils import data
from torch.optim.adadelta import Adadelta
import sys
sys.path.append('/kaggle/working/ST-GCN-Pytorch')
from models.stgcn import *
from torch.utils.data import DataLoader, TensorDataset
from collections import OrderedDict
import logging
import yaml
from dataloader.dataset import processing_data
import datetime

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# clear memory cuda
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

# Get parameter
with open("/kaggle/input/yaml-action-pkl/config.yaml", "r") as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print("Error loading YAML:", exc)
        config = {}

# parameter
input_dataset_train = config.get('dataset-path-train', '')
input_dataset_test = config.get('dataset-path-test', '')
epochs = config.get('epochs', 50)
batch_size = config.get('batch-size', 32)
input_size = config.get('img-size', 224)
num_frame = config.get('num-frame', 15)
path_save_model = config.get('project', 'saved_models')
classes_name = config.get('classes', ['Fall Down', 'Lying Down', 'Sit down', 'Sitting', 'Stand up', 'Standing', 'Walking'])

print("Class name:", classes_name)

features, labels = [], []
# Load dataset train
with open(input_dataset_train, 'rb') as f:
    fts, lbs = pickle.load(f)
    features.append(fts)
    labels.append(lbs)
del fts, lbs

# Normalize class
labels = np.concatenate(labels, axis=0)
features = np.concatenate(features, axis=0)
features = features[:, ::2, :, :]
features[:, :, :, :2] = processing_data(features[:, :, :, :2])
x_train = features
y_train = labels

features, labels = [], []
with open(input_dataset_test, 'rb') as f:
    fts, lbs = pickle.load(f)
    features.append(fts)
    labels.append(lbs)
del fts, lbs

labels = np.concatenate(labels, axis=0)
features = np.concatenate(features, axis=0)
features = features[:, ::2, :, :]
features[:, :, :, :2] = processing_data(features[:, :, :, :2])
x_valid = features
y_valid = labels

del features, labels

train_dataset = TensorDataset(torch.tensor(x_train, dtype=torch.float32).permute(0, 3, 1, 2),
                              torch.tensor(y_train, dtype=torch.float32))
val_dataset = TensorDataset(torch.tensor(x_valid, dtype=torch.float32).permute(0, 3, 1, 2),
                            torch.tensor(y_valid, dtype=torch.float32))

del x_train, x_valid, y_train, y_valid

# create folder save
if not os.path.exists(path_save_model):
    os.mkdir(path_save_model)
count = 0
while os.path.exists(path_save_model + f'/exp{count}'):
    count += 1
path_save_model = path_save_model + f'/exp{count}'
os.mkdir(path_save_model)

# load data loader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=batch_size, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=batch_size, pin_memory=True)

del train_dataset, val_dataset

graph_args = {'strategy': 'spatial'}
model = TwoStreamSpatialTemporalGraph(graph_args, len(classes_name)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
losser = torch.nn.BCELoss()

def main():
    model_trained = train_model(model, losser, optimizer, num_epochs=epochs)
    torch.save(model_trained.state_dict(), path_save_model + '/last.pt')
    logging.warning('Saved last model at {}'.format(path_save_model))
    print("Complete !")

if __name__ == '__main__':
    main()


Class name: ['Fall Down', 'Lying Down', 'Sit down', 'Sitting', 'Stand up', 'Standing', 'Walking']


Epoch 49: 100%|██████████| 887/887 [01:40<00:00,  8.84batch/s]


Complete !
